In [19]:
import requests
import json
from configuration import api_key
import pandas as pd
from pathlib import Path

from pandas import json_normalize 




park_visitors_csv_file = Path("../Resources/national_park_visitors.csv")

park_visitors_data= pd.read_csv(park_visitors_csv_file)

park_visitors_df = pd.DataFrame(park_visitors_data)

#park_visitors_list = park_visitors_data['ParkName'].to_numpy()

#park_visitors_list

endpoint = f"https://developer.nps.gov/api/v1/parks?limit=600&api_key={api_key}"

parks_data = requests.get(endpoint).json()

#print(json.dumps(parks_data['data'], indent=4, sort_keys=True))
#df = pd.read_json(parks_data['data'][0]['name'])
#parks_data['data'][0]['name'].info()


normalized_parks_data = json_normalize(parks_data, 'data')


all_parks_df = pd.DataFrame(normalized_parks_data).drop(columns=[
'id', 
    'url', 
    'latLong', 
    'directionsInfo', 
    'directionsUrl', 
    'addresses', 
    'images', 
    'contacts.phoneNumbers',
    'contacts.emailAddresses'
    
])

national_parks_df = all_parks_df.loc[(all_parks_df['designation'] == "National Park") | (all_parks_df['designation'] == "National Park & Preserve")]

#national_parks_df.head()
#national_parks_df['activities'][0]
#print(park_visitors_df)
#all_parks_df.head(50)

KeyError: 0